In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import os.path as op
import os

import seaborn as sns
sns.set(context="notebook", style="whitegrid", font_scale=1.2)

import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline


In [58]:
data = pd.read_csv("visual-Basketball.csv")
data.head()
df2=data.copy()
# read particular columns
df2 = df2[['ID','condition','condition_o','trial_r', 'QED_r' ,'FirstGaze_r',
         'QEstart_r','QEend_r','Release_r',
         'performance_r','FirstGaze_o','QED_o',
         'QEstart_o','QEend_o','Release_o','performance_o' ]]  
# convert data to long format
df_2 = pd.melt(df2, id_vars=['ID','trial_r', 'QED_r' ,'FirstGaze_r','QEstart_r',
                            'QEend_r','Release_r','performance_r','FirstGaze_o','QED_o',
                            'QEstart_o','QEend_o','Release_o','performance_o' ],
               value_vars=['condition','condition_o'], var_name ='con', value_name ='condition0')

# add some columns based on the pre or post condition
df_2['start'] = np.where(df_2.condition0 =='pre',df_2.FirstGaze_r,df_2.FirstGaze_o)
df_2['Perform'] = np.where(df_2.condition0 =='pre',df_2.performance_r,df_2.performance_o)
df_2['Release'] = np.where(df_2.condition0 =='pre',df_2.Release_r,df_2.Release_o)
df_2['QED'] = np.where(df_2.condition0 =='pre',df_2.QED_r,df_2.QED_o)
df_2.rename(columns={'condition0':'condition'}, inplace=True)
df_2.rename(columns={'trial_r':'trial'}, inplace=True)
df_2 = df_2[['ID','trial','condition', 'start' ,'Release','QED','Perform']]

df = df_2.copy()

dataframe0= {'Id':[],
            'trial':[],
            'condition' :[],
           'numb_fix':[],
           'fix_mean_dur':[],
           'fix_std_dur':[] ,
            'QED':[],
           'sccades_dur':[],
           'microsac_dur':[] ,
           'sccades_mean_vel':[],
           'microsaccedes_vel':[]
          }
df_last0 = pd.DataFrame(dataframe0)
df

,ID,trial,condition,start,Release,QED,Perform
0,1,1,pre,1.0,161.0,564.0,1
1,1,2,pre,61.0,131.0,332.0,2
2,1,3,pre,20.0,151.0,298.0,1
3,1,4,pre,46.0,151.0,365.0,5
4,1,5,pre,1.0,126.0,132.0,2
...,...,...,...,...,...,...,...
715,36,6,post,39.0,168.0,100.0,3
716,36,7,post,1.0,164.0,436.0,3
717,36,8,post,1.0,122.0,1722.0,1
718,36,9,post,1.0,156.0,2086.0,4


In [59]:
def cart_to_spherical(data, apply_rad2deg=False):
    # convert to spherical coordinates
    # source: http://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion
    x = data.gaze_point_3d_x
    y = data.gaze_point_3d_y
    z = data.gaze_point_3d_z
    r = np.sqrt(x ** 2 + y ** 2 + z ** 2)
    theta = np.arccos(y / r)  # for elevation angle defined from Z-axis down
    psi = np.arctan2(z, x)
    
    if apply_rad2deg:
        theta = np.rad2deg(theta)
        psi = np.rad2deg(psi)
    
    return r, theta, psi

In [60]:
for id in range(1, 37):
    for tr in range(10):
        ID = id  # subject ID - a numer beteween 1 and 36
        condition = 'post'   # condition post or pre
        trial = tr   # trial between 1 to 10
        ID_str = str(ID)  
        trial_s = str(trial) 
        trial_str = "00"+trial_s+".csv"   # trial string
        file_type = 'fixation' # fixation file
        data_folder = ".\gaze_data"
        raw_fix= op.join(data_folder,ID_str,condition,file_type,trial_str)
        file_type_g = 'gaze position'  # gaze file
        raw_gaze = op.join(data_folder,ID_str,condition,file_type_g,trial_str)
        df_fix = pd.read_csv (raw_fix)
        df_fix = df_fix[['start_frame_index','end_frame_index','confidence']]
        df_gaze = pd.read_csv (raw_gaze)
#         print(raw_fix)
#         print(raw_gaze)
        release = np.array(df[(df.ID ==ID) & (df.condition == condition) & (df.trial == trial+1)].Release)
        start =  np.array(df[(df.ID ==ID) & (df.condition == condition) & (df.trial == trial+1)].start)
        performance =  np.array(df[(df.ID ==ID) & (df.condition == condition) & (df.trial == trial+1)].Perform)  #performance
        QED =  np.array(df[(df.ID ==ID) & (df.condition == condition) & (df.trial == trial+1)].QED)  #performance
        performance =performance[0]
        QED =QED[0]
        #print(QED)

        df_fix_c= df_fix[(df_fix.start_frame_index >=start[0]) & (df_fix.start_frame_index <= release[0])]
        df_gaze_c= df_gaze[(df_gaze.world_index >=start[0]) & (df_gaze.world_index <= release[0])]
        exported_gaze =df_gaze_c
        exported_gaze.gaze_point_3d_z
        negative_z_mask = exported_gaze.gaze_point_3d_z < 0
        negative_z_values = exported_gaze.loc[negative_z_mask,["gaze_point_3d_z"]]
        exported_gaze.loc[negative_z_mask, ["gaze_point_3d_z"]] = negative_z_values * -1
        r, theta, psi = cart_to_spherical(exported_gaze, apply_rad2deg=True)
        squared_theta_diff = np.diff(theta) ** 2
        squared_psi_diff = np.diff(psi) ** 2
        deg_diff = np.sqrt(squared_theta_diff + squared_psi_diff)
        ts_diff = np.diff(exported_gaze.world_timestamp)
        deg_per_sec = deg_diff / ts_diff

        # define saccade as velociyt higher than 20 degree per sec and lower than 30
        sccades = deg_per_sec[(deg_per_sec > 19) & (deg_per_sec < 31)] 
         # microsaccades are gaze movement higher than 30 degree per sec
        microsaccedes = deg_per_sec[(deg_per_sec > 31)]

        # number of saccades and microsaccades
        sccades_duration =  len(sccades)*16
        microsaccedes_duration = len(microsaccedes)*16

        # mean velocity of saccades
        sccades_vel = sccades.mean()
        microsaccedes_vel = microsaccedes.mean()
        fix_number = len(df_fix_c)
        df_fix_c['duration'] = (df_fix_c.end_frame_index - df_fix_c.start_frame_index)*16
        fix_mean_duration = df_fix_c.duration.mean()
        fix_std_duration = round(df_fix_c.duration.std(),2)
        
        dataframe= {'Id':[ID],
            'trial':[trial+1],
            'condition' : [condition],
           'numb_fix': [fix_number],
           'fix_mean_dur': [fix_mean_duration],
           'fix_std_dur' :[fix_std_duration],
           'QED':[QED],
           'sccades_dur' :[sccades_duration],
           'microsac_dur' : [microsaccedes_duration],
           'sccades_mean_vel' :[sccades_vel],
           'microsaccedes_vel' :[microsaccedes_vel],
           'performance' : [performance]
           
          }
 
# Create DataFrame
        df_last = pd.DataFrame(dataframe)
        df_last0=df_last0.append(df_last)
        print(tr)
        print(id)


    

    

C:\Users\Reza Saeedpour\anaconda3\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
<ipython-input-60-8fcb3325da23>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fix_c['duration'] = (df_fix_c.end_frame_index - df_fix_c.start_frame_index)*16
C:\Users\Reza Saeedpour\anaconda3\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

0
1
1
1
2
1
3
1
4
1
5
1
6
1
7
1
8
1
9
1
0
2
1
2
2
2


C:\Users\Reza Saeedpour\anaconda3\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
<ipython-input-60-8fcb3325da23>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fix_c['duration'] = (df_fix_c.end_frame_index - df_fix_c.start_frame_index)*16
C:\Users\Reza Saeedpour\anaconda3\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

3
2
4
2
5
2
6
2
7
2
8
2
9
2
0
3
1
3
2
3
3
3
4
3


<ipython-input-60-8fcb3325da23>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fix_c['duration'] = (df_fix_c.end_frame_index - df_fix_c.start_frame_index)*16
C:\Users\Reza Saeedpour\anaconda3\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
<ipython-input-60-8fcb3325da23>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

...

9
33
0
34
1
34
2
34
3
34
4
34
5
34
6
34
7
34
8
34
9
34


C:\Users\Reza Saeedpour\anaconda3\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
<ipython-input-60-8fcb3325da23>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fix_c['duration'] = (df_fix_c.end_frame_index - df_fix_c.start_frame_index)*16
C:\Users\Reza Saeedpour\anaconda3\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

0
35
1
35
2
35
3
35
4
35
5
35
6
35
7
35
8
35
9
35


<ipython-input-60-8fcb3325da23>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fix_c['duration'] = (df_fix_c.end_frame_index - df_fix_c.start_frame_index)*16
C:\Users\Reza Saeedpour\anaconda3\lib\site-packages\pandas\core\indexing.py:1787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
<ipython-input-60-8fcb3325da23>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

0
36
1
36
2
36
3
36
4
36
5
36
6
36
7
36
8
36
9
36


In [62]:
df_last0

df_last0.to_csv('result_post.csv', mode = 'a', header = True, index=False)